In [9]:
from collections import defaultdict, deque

N = 11


def get_num_digits_to_possible_solutions(generator, max_num_digits):
    num_digits_to_possible_solutions = defaultdict(list)
    while True:
        possible_solution = next(generator)
        num_digits = len(str(possible_solution))
        if num_digits < 2:
            continue
        if num_digits > max_num_digits:
            break
        num_digits_to_possible_solutions[num_digits].append(possible_solution)
    return num_digits_to_possible_solutions


def get_numbers_with_digit_sum(target_sum):
    queue = deque([(0, 0)])  # each tuple contains (current_number, current_sum)

    while queue:
        current_number, current_sum = queue.popleft()

        # Start the digit range from 1 if current_number is 0 to avoid leading zeros
        start_digit = 1 if current_number == 0 else 0

        for digit in range(start_digit, 10):
            next_number = current_number * 10 + digit
            next_sum = current_sum + digit
            if next_sum > target_sum:
                break  # No further numbers need to be checked beyond this digit
            elif next_sum == target_sum:
                yield next_number
            else:
                queue.append((next_number, next_sum))


def get_fibonacci():
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b


def get_squares():
    i = 1
    while True:
        yield i**2
        i += 1


def get_multiples_of_thirty_seven():
    i = 1
    while True:
        yield 37 * i
        i += 1


def get_powers_of_seven():
    i = 1
    while True:
        yield 7**i
        i += 1

In [10]:
n_by_one_shadings = []


def get_all_binary_strings(n):
    return [format(i, f"0{n}b") for i in range(2**n)]


def filter_binary_strings(strings):
    return [
        s
        for s in strings
        if "101" not in s and "11" not in s and s[:2] != "01" and s[-2:] != "10"
    ]


n_by_one_shadings = filter_binary_strings(get_all_binary_strings(N))
print(len(n_by_one_shadings))
print(n_by_one_shadings)

54
['00000000000', '00000000001', '00000000100', '00000001000', '00000001001', '00000010000', '00000010001', '00000100000', '00000100001', '00000100100', '00001000000', '00001000001', '00001000100', '00001001000', '00001001001', '00010000000', '00010000001', '00010000100', '00010001000', '00010001001', '00010010000', '00010010001', '00100000000', '00100000001', '00100000100', '00100001000', '00100001001', '00100010000', '00100010001', '00100100000', '00100100001', '00100100100', '10000000000', '10000000001', '10000000100', '10000001000', '10000001001', '10000010000', '10000010001', '10000100000', '10000100001', '10000100100', '10001000000', '10001000001', '10001000100', '10001001000', '10001001001', '10010000000', '10010000001', '10010000100', '10010001000', '10010001001', '10010010000', '10010010001']


In [11]:
num_digits_to_digit_sum_sevens = get_num_digits_to_possible_solutions(
    get_numbers_with_digit_sum(7), N
)
# num_digits_to_powers_of_sevens = get_num_digits_to_possible_solutions(get_powers_of_seven(), N)
# print(num_digits_to_powers_of_sevens)
num_digits_to_fibonacci = get_num_digits_to_possible_solutions(get_fibonacci(), N)
num_digits_to_squares = get_num_digits_to_possible_solutions(get_squares(), N)

In [12]:
import itertools


def get_possible_solutions(
    num_digits_to_possible_solutions, binary_string, underlying_region
):
    # The binary string divides the underlying region into shaded and unshaded regions
    # We have to fill in each consequtive array of 0s with a number from the possible solutions, such that
    # each underlying region in a given array of 0s has the same digit
    # Eg. if the binary string is "1001" and the underlying region is "aabb", then we need 2 digit numbers where
    # the first digit is different from the second digit to fill in the 0s
    zero_regions = []
    current_region = ""
    for i, bit in enumerate(binary_string):
        if bit == "0":
            current_region += underlying_region[i]
        else:
            if current_region:
                zero_regions.append(current_region)
                current_region = ""
    if current_region:
        zero_regions.append(current_region)

    # This will be a 2D array of possible solutions for each region of 0s, we can mix and match these to get the final solution
    assignments = []
    for region in zero_regions:
        possible_assignments = []
        num_digits = len(region)
        possible_nums = num_digits_to_possible_solutions[num_digits]
        for num in possible_nums:
            can_fit = True
            str_num = str(num)
            for i in range(len(region) - 1):
                j = i + 1
                if region[i] == region[j] and str_num[i] != str_num[j]:
                    can_fit = False
                elif region[i] != region[j] and str_num[i] == str_num[j]:
                    can_fit = False
            if can_fit:
                possible_assignments.append(num)

            # regions_to_digits = defaultdict(set)
            # for i, c in enumerate(region):
            #     regions_to_digits[c].add(str(num)[i])
            # digit_set = set(str(num))
            # region_set = set(region)
            # if all(len(digits) == 1 for digits in regions_to_digits.values()) and len(digit_set) == len(region_set):
            #     possible_assignments.append(num)
        assignments.append(possible_assignments)
    # Now we have to mix and match the possible assignments to get the final solution
    total_assignments = list(itertools.product(*assignments))
    possible_solutions = []
    for assignment in total_assignments:
        joined_assignment = "".join(str(num) for num in assignment)
        solution = ""
        i = 0
        for bit in binary_string:
            if bit == "0":
                solution += joined_assignment[i]
                i += 1
            else:
                solution += "*"
        possible_solutions.append(solution)
    return possible_solutions

In [13]:
def get_solutions(num_digits_to_possible_solutions, underlying_region):
    solutions = []
    for shading in n_by_one_shadings:
        possible_solutions = get_possible_solutions(
            num_digits_to_possible_solutions, shading, underlying_region
        )
        if possible_solutions:
            solutions += possible_solutions
    return solutions


digit_sum_sevens_underlying_region = "abbccddefff"
digit_sum_sevens_solutions = get_solutions(num_digits_to_digit_sum_sevens, digit_sum_sevens_underlying_region)
digit_sum_sevens_solutions = [(a, a) for a in digit_sum_sevens_solutions]
print(digit_sum_sevens_solutions)

fibonacci_underlying_region = "abccddefgih"
# fibonacci_underlying_region = "abcdd"
fibonacci_solutions = get_solutions(num_digits_to_fibonacci, fibonacci_underlying_region)
fibonacci_solutions = [(a, a) for a in fibonacci_solutions]
print(fibonacci_solutions)

squares2_underlying_region = "abbbbbbbccd"
squares2_solutions = get_solutions(num_digits_to_squares, squares2_underlying_region)
squares2_solutions = [(a, a) for a in squares2_solutions]
print(squares2_solutions)


[('1001100211*', '1001100211*'), ('100114*1222', '100114*1222'), ('100114*4111', '100114*4111'), ('122002*1222', '122002*1222'), ('122002*4111', '122002*4111'), ('200113*1222', '200113*1222'), ('200113*4111', '200113*4111'), ('200221*1222', '200221*1222'), ('200221*4111', '200221*4111'), ('211003*1222', '211003*1222'), ('211003*4111', '211003*4111'), ('300112*1222', '300112*1222'), ('300112*4111', '300112*4111'), ('311002*1222', '311002*1222'), ('311002*4111', '311002*4111'), ('411001*1222', '411001*1222'), ('411001*4111', '411001*4111'), ('100114*133*', '100114*133*'), ('100114*322*', '100114*322*'), ('100114*511*', '100114*511*'), ('122002*133*', '122002*133*'), ('122002*322*', '122002*322*'), ('122002*511*', '122002*511*'), ('200113*133*', '200113*133*'), ('200113*322*', '200113*322*'), ('200113*511*', '200113*511*'), ('200221*133*', '200221*133*'), ('200221*322*', '200221*322*'), ('200221*511*', '200221*511*'), ('211003*133*', '211003*133*'), ('211003*322*', '211003*322*'), ('21100

In [14]:
def merge(first_solutions, second_solutions, compatibility):
    new_solutions = []
    for i in range(len(first_solutions)):
        first_top, first_bottom = first_solutions[i]
        for j in range(len(second_solutions)):
            second_top, second_bottom = second_solutions[j]
            can_merge = True
            for a,b,c in zip(first_bottom, second_top, compatibility):
                if a == "*" and b == "*":
                    can_merge = False
                elif a != "*" and b != "*":
                    if c == "1":
                        can_merge &= a == b
                    else:
                        can_merge &= a != b
            if can_merge:
                new_solutions.append((first_top, second_bottom))
    return new_solutions

merged_solutions = merge(digit_sum_sevens_solutions, fibonacci_solutions, "11010011101")

merged_solutions = merge(merged_solutions, squares2_solutions, "10001101010")
print(merged_solutions)

[('133*100411*', '1444*444889')]


In [36]:
# To proceed, we cannot do what we've done. There are too many multiples of 37.
# Instead, since we have the actual solution, and only 54 valid shadings, we can proceed line by line brute forcing it.

square2_solution = merged_solutions[0][1]
print(square2_solution)

compatibility = "01111001111"
next_solutions = []
for shading in n_by_one_shadings:
    # do things that don't scale lol
    if shading[4] == "1":
        continue

    solution = shading.replace("1", "*")
    solution = solution.replace("0", "?")
    solution = list(solution)
    for i in range(N):
        if compatibility[i] == "1":
            solution[i] = square2_solution[i]
    next_solutions.append(solution)
print(len(next_solutions))
print(max(solution.count("?") for solution in next_solutions))
print(next_solutions[0])



1444*444889
44
3
['?', '4', '4', '4', '*', '?', '?', '4', '8', '8', '9']


In [38]:
def check_string(s):
    # This will be a string of digits and *s
    # We want to check that every contiguous region of digits is a multiple of 37

    current_region = ""
    for c in s:
        if c == "*":
            if current_region:
                if int(current_region) % 37 != 0:
                    return False
                current_region = ""
        else:
            current_region += c
    if current_region:
        if int(current_region) % 37 != 0:
            return False
    return True

def generate_solutions(template):
    question_mark_indices = [i for i, c in enumerate(template) if c == "?"]
    if len(question_mark_indices) == 0:
        return template
    else:
        assignments = itertools.product("0123456789", repeat=len(question_mark_indices))
        for assignment in assignments:
            new_template = template[:]
            for i, index in enumerate(question_mark_indices):
                new_template[index] = assignment[i]
            new_template = "".join(new_template)
            if check_string(new_template):
                yield new_template

print(list(generate_solutions(next_solutions[0])))

['0444*184889', '0444*554889', '0444*924889']
